In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tqdm.keras import TqdmCallback
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pickle
import unicodedata
from sklearn.metrics import classification_report

# ===============================================
# CONFIGURASI UTAMA
# ===============================================
# PASTIKAN FILE PATH SESUAI DENGAN NAMA FILE ANDA
FILE_PATH = "production_judol_detection_old.csv" 
MAX_WORDS = 20000
MAX_LEN = 100
EMBED_DIM = 128
BATCH_SIZE = 64
EPOCHS = 25
LR = 1e-3
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
TEST_SIZE_FINAL = 0.20
NEW_ALPHA = 0.9 # Alpha untuk Focal Loss, memberi bobot lebih ke kelas Judol (1)
OPTIMAL_THRESHOLD = 0.65 
TEXT_COLUMN = 'cleaned_comment_text'
TARGET_COLUMN = 'target'

# ===============================================
# 1. DATA AUGMENTATION: SINTESIS JUDOL BARU
# ===============================================

# Pola Brand & Kata Kunci
prefixes = ['slot', 'jp', 'wd', 'depo', 'gacor', 'raja', 'bosku', 'mantap']
numbers = [str(i) for i in range(100, 999, 100)] + [str(i) for i in range(11, 99, 11)]
phrases = ['gacor banget di sini!', 'pasti jp di situs ini', 'wd cepat tanpa ribet', 'cair terus bosku']

synthetic_data = []
for i in range(1200): 
    prefix = np.random.choice(prefixes)
    num = np.random.choice(numbers)
    phrase = np.random.choice(phrases)
    
    brand_name = prefix + num
    comment = f"{phrase} coba main di {brand_name} aja deh"
    
    # Cleaning dasar untuk data sintetis
    cleaned_comment = re.sub(r'\s+', ' ', re.sub(r"[^a-z0-9\s]", " ", comment.lower())).strip()
    
    synthetic_data.append({
        "comment_text": comment,
        "target": 1,
        "cleaned_comment_text": cleaned_comment,
        "combined_text": cleaned_comment + " " + comment 
    })

df_synthetic = pd.DataFrame(synthetic_data)
df_synthetic['cleaned_comment_text'] = df_synthetic['cleaned_comment_text'].str.replace('coba main di ', '').str.replace(' aja deh', '')

# ===============================================
# 2. LOAD & GABUNGKAN DATA
# ===============================================

def merge_spaced_characters(text):
    """
    Menggabungkan pola huruf/angka yang dipisah spasi namun merupakan satu kata.
    Contoh: 'p u l a u w i n 8 8' -> 'pulauwin88'
    """
    tokens = text.split()
    merged_tokens = []
    buffer = []

    def flush_buffer():
        if len(buffer) >= 3:  # Minimal 3 karakter untuk dianggap kata terpisah
            merged_tokens.append("".join(buffer))
        else:
            merged_tokens.extend(buffer)
        buffer.clear()

    for tok in tokens:
        if len(tok) == 1 and re.match(r"[a-zA-Z0-9]", tok):
            buffer.append(tok.lower())
        else:
            flush_buffer()
            merged_tokens.append(tok)

    flush_buffer()
    return " ".join(merged_tokens)

def clean_text_prep(text):
    """Fungsi cleaning yang digunakan untuk Tokenisasi."""
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    
    text = merge_spaced_characters(text)
    
    # NFKD: Mengubah karakter fancy (misal 𝗛𝗢𝗧𝐉𝐄𝐏𝐄𝟱𝟱) menjadi bentuk dasarnya (HOTJEPE55)
    text = unicodedata.normalize('NFKD', text)
    
    # Encode/Decode: Menghapus semua karakter non-ASCII (seperti emoji 🀄🥰, diakritik)
    # yang tersisa setelah NFKD, hanya menyisakan karakter dasar ASCII.
    # Ini adalah langkah kunci untuk mengatasi masalah hilangnya karakter fancy.
    text = text.encode('ascii', 'ignore').decode('utf-8')
    
    # 3. Lowercase (Dapat dilakukan setelah konversi ASCII)
    text = text.lower()
    
    # 4. Hapus sisa karakter yang bukan a-z, 0-9, atau spasi
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # 5. Rapikan spasi
    text = re.sub(r"\s+", " ", text).strip()
    return text

print("--- 1. Memuat Data, Augmentasi, dan Preprocessing ---")
try:
    df_original = pd.read_csv(FILE_PATH)
    # Pastikan kolom utama ada dan target numerik
    df_original = df_original.dropna(subset=[TEXT_COLUMN, TARGET_COLUMN])
    df_original[TARGET_COLUMN] = df_original[TARGET_COLUMN].astype(int)
    
    # Gabungkan data asli dengan data sintetik
    df_combined = pd.concat([df_original[['comment_text', 'cleaned_comment_text', 'target', 'combined_text']], df_synthetic], ignore_index=True)

    # Bersihkan Teks pada kolom fitur utama
    df_combined[TEXT_COLUMN] = df_combined[TEXT_COLUMN].fillna('').apply(clean_text_prep)
    
    # Filter data yang teksnya tidak kosong
    df_combined = df_combined[df_combined[TEXT_COLUMN] != '']

except FileNotFoundError:
    print(f"❌ ERROR: File '{FILE_PATH}' tidak ditemukan.")
    exit()

X = df_combined[TEXT_COLUMN]
Y = df_combined[TARGET_COLUMN]

print(f"Total data setelah augmentasi dan cleaning: {len(X)} baris.")
print(f"Jumlah Judol (1) sebelum split: {Y.sum()}")

# ===============================================
# 3. SPLIT DATA (STRATIFIED)
# ===============================================
print("\n--- 2. Split Data (80% Train, 20% Test Stratified) ---")
# Pembagian 80% Train, 20% Test Murni
X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    X, 
    Y, 
    test_size=TEST_SIZE_FINAL, 
    random_state=SEED, 
    stratify=Y 
)

# ===============================================
# 4. FEATURE EXTRACTION (Tokenisasi & Padding)
# ===============================================
print("\n--- 3. Feature Extraction: Tokenisasi dan Padding ---")
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text.astype(str))

# SIMPAN TOKENIZER
with open('tokenizer_augmented.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Ubah Data Latih menjadi Vektor Fitur
X_train_seq = tokenizer.texts_to_sequences(X_train_text.astype(str))
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Ubah Data Uji (Murni) menjadi Vektor Fitur
X_test_seq = tokenizer.texts_to_sequences(X_test_text.astype(str))
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')


# ===============================================
# 5. TERAPKAN SMOTE (HANYA PADA DATA LATIH)
# ===============================================
print("\n--- 4. Terapkan SMOTE (Oversampling pada Data Latih) ---")
Y_train_np = Y_train.values 
smote = SMOTE(sampling_strategy='minority', random_state=SEED)

# SMOTE diterapkan pada fitur numerik (X_train_padded) dan labelnya (Y_train_np)
X_resampled, Y_resampled = smote.fit_resample(X_train_padded, Y_train_np)

# Hasil Akhir (Data Latih yang Sudah Seimbang)
X_train_final = np.array(X_resampled)
Y_train_final = np.array(Y_resampled)

# Verifikasi Rasio Kelas
judol_count = Y_train_final.sum()
non_judol_count = len(Y_train_final) - judol_count
print(f"✅ Data Latih Akhir Setelah SMOTE: {len(X_train_final)} baris (Rasio 1:{round(non_judol_count / judol_count)})")


# ===============================================
# 6. TRAINING DENGAN DATA SMOTE
# ===============================================
def focal_loss(gamma=2., alpha=NEW_ALPHA):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        loss = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                 - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return loss
    
model_aug = Sequential([
    Embedding(MAX_WORDS, EMBED_DIM, input_length=MAX_LEN),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
    GlobalMaxPool1D(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(1, activation="sigmoid")
])

model_aug.compile(loss=focal_loss(), optimizer=Adam(learning_rate=LR), metrics=["accuracy"])

callbacks = [
    EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
    TqdmCallback(verbose=1),
]

print(f"\nMemulai Pelatihan Model dengan Data SMOTE (Alpha={NEW_ALPHA})...")

history = model_aug.fit(
    X_train_final, Y_train_final, # <-- MENGGUNAKAN DATA HASIL SMOTE
    validation_split=0.1, # Menggunakan 10% dari data SMOTE untuk validasi
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    # class_weight TIDAK DIPERLUKAN KARENA DATA SUDAH SEIMBANG OLEH SMOTE
    callbacks=callbacks,
    verbose=0
)

# SAVE MODEL BARU
NEW_MODEL_AUG_PATH = "judol_detection_augmented_smote.h5"
model_aug.save(NEW_MODEL_AUG_PATH)
print(f"\nModel baru disimpan sebagai {NEW_MODEL_AUG_PATH} ✅")


# ===============================================
# 7. EVALUASI DAN UJI MODEL BARU (KODE PERBAIKAN)
# ===============================================

print("\n=============================================")
print("📊 EVALUASI AKHIR MODEL PADA DATA UJI MURNI")
print("=============================================")

# FIX: Konversi Y_test (Pandas Series) menjadi NumPy Array untuk menghindari ValueError
Y_test_np = Y_test.values 

# Evaluasi Model pada Data Uji Murni (X_test_padded)
loss, accuracy = model_aug.evaluate(X_test_padded, Y_test_np, verbose=0) 

print(f"Loss pada Data Uji: {loss:.4f}")
print(f"Akurasi pada Data Uji: {accuracy:.4f}")

# Hitung Precision, Recall, dan F1-Score menggunakan OPTIMAL_THRESHOLD
y_pred_proba = model_aug.predict(X_test_padded, verbose=0)
y_pred_classes = (y_pred_proba > OPTIMAL_THRESHOLD).astype(int)

print(f"\nLaporan Klasifikasi (Threshold={OPTIMAL_THRESHOLD}):")
print(classification_report(Y_test_np, y_pred_classes)) # Menggunakan Y_test_np


# ===============================================
# 8. UJI MANUAL PADA KASUS SULIT (Pure ML)
# ===============================================

def clean_text(text):
    """Fungsi cleaning yang digunakan saat prediksi (dengan normalisasi Unicode)"""
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Normalisasi Unicode
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') 
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def predict_text_ml_only(text, model, tokenizer, threshold=OPTIMAL_THRESHOLD):
    """Melakukan prediksi Judol murni tanpa Heuristik."""
    
    text_clean = clean_text(text)
    
    seq = tokenizer.texts_to_sequences([text_clean])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    
    pred_prob = model.predict(pad, verbose=0)[0][0]
    is_judol = pred_prob > threshold
    
    label = "🔥 **JUDOL (SMOTE ML)**" if is_judol else "✅ Bukan Judol (SMOTE ML)"
    
    print(f"---")
    print(f"💬 Input: '{text}'")
    print(f"🧹 Clean: '{text_clean}'")
    print(f"🎯 Prediksi: {label} (Probabilitas: {pred_prob:.3f} | T={threshold})")

manual_data = [
    "WD 15 juta di BOSKU777 langsung cair tanpa ribet!", 
    "buruan daftar di MANTAPJEPE888 promonya gede banget", 
    "gacor di situs ini, main di kaisar999 depo 50 ribu wd 500 ribu", 
    "main di mahadewa pasti cair",
    "Ini brand baru: GACORWD77. Pasti JP!", 
    "Coba aja main di brand baru SLOTBARU555, depo langsung wd", 
    "nonton film gratis di youtube channel ini aja",
]

print("\n=============================================")
print(f"🚀 UJI COBA MODEL MURNI ML (SMOTE Trained, Threshold={OPTIMAL_THRESHOLD})")
print("=============================================")
for data in manual_data:
    predict_text_ml_only(data, model_aug, tokenizer)

--- 1. Memuat Data, Augmentasi, dan Preprocessing ---
Total data setelah augmentasi dan cleaning: 20444 baris.
Jumlah Judol (1) sebelum split: 2732

--- 2. Split Data (80% Train, 20% Test Stratified) ---

--- 3. Feature Extraction: Tokenisasi dan Padding ---

--- 4. Terapkan SMOTE (Oversampling pada Data Latih) ---
✅ Data Latih Akhir Setelah SMOTE: 28338 baris (Rasio 1:1)


/home/wtf/anaconda3/lib/python3.13/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]


Memulai Pelatihan Model dengan Data SMOTE (Alpha=0.9)...

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.



Model baru disimpan sebagai judol_detection_augmented_smote.h5 ✅

📊 EVALUASI AKHIR MODEL PADA DATA UJI MURNI
Loss pada Data Uji: 0.1088
Akurasi pada Data Uji: 0.5057

Laporan Klasifikasi (Threshold=0.65):
              precision    recall  f1-score   support

           0       0.99      0.57      0.73      3543
           1       0.26      0.97      0.41       546

    accuracy                           0.63      4089
   macro avg       0.62      0.77      0.57      4089
weighted avg       0.89      0.63      0.68      4089


🚀 UJI COBA MODEL MURNI ML (SMOTE Trained, Threshold=0.65)
---
💬 Input: 'WD 15 juta di BOSKU777 langsung cair tanpa ribet!'
🧹 Clean: 'wd 15 juta di bosku777 langsung cair tanpa ribet'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.994 | T=0.65)
---
💬 Input: 'buruan daftar di MANTAPJEPE888 promonya gede banget'
🧹 Clean: 'buruan daftar di mantapjepe888 promonya gede banget'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.989 | T=0.65)
---
💬 Input: 'gacor d

In [11]:
# file: test_robust_judol_model.py

import os
import re
import unicodedata
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# =========================
# CONFIG
# =========================
MAX_LEN = 100
NEW_ALPHA = 0.51
OPTIMAL_THRESHOLD = 0.65

TOKENIZER_PATH = "tokenizer_augmented_robust.pickle"
MODEL_PATH = "judol_detection_augmented_smote_robust.keras"

ZERO_WIDTH_CHARS = [
    '\u200b', '\u200c', '\u200d', '\u2060', '\uFEFF'
]

SEP_CHARS = r"[\s\.\-_\|•·\u2063]"
MIN_MERGE_LEN = 3


# =========================
# CLEANING PIPELINE (SAMA PERSIS DENGAN TRAINING)
# =========================

def remove_zero_width(text: str) -> str:
    for z in ZERO_WIDTH_CHARS:
        text = text.replace(z, '')
    return text

def normalize_unicode(text: str) -> str:
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def merge_spaced_characters_simple(text: str, min_len=MIN_MERGE_LEN) -> str:
    tokens = text.split()
    merged = []
    buffer = []
    for t in tokens:
        if len(t) == 1 and re.match(r'[A-Za-z0-9]', t):
            buffer.append(t)
        else:
            if len(buffer) >= min_len:
                merged.append("".join(buffer))
            elif buffer:
                merged.extend(buffer)
            buffer = []
            merged.append(t)
    if buffer:
        if len(buffer) >= min_len:
            merged.append("".join(buffer))
        else:
            merged.extend(buffer)
    return " ".join(merged)

def merge_obfuscated_sequences(text: str, min_len=MIN_MERGE_LEN) -> str:
    tokens = re.split(r'(\s+)', text)
    out_tokens = []
    buffer = []
    for tok in tokens:
        if tok.isspace():
            if buffer:
                merged = re.sub(SEP_CHARS, '', "".join(buffer))
                out_tokens.append(merged)
                buffer = []
            out_tokens.append(tok)
            continue
        if re.fullmatch(r'(?:[A-Za-z0-9]|' + SEP_CHARS + r')+$', tok):
            buffer.append(tok)
        else:
            if buffer:
                merged = re.sub(SEP_CHARS, '', "".join(buffer))
                out_tokens.append(merged)
                buffer = []
            out_tokens.append(tok)
    if buffer:
        merged = re.sub(SEP_CHARS, '', "".join(buffer))
        out_tokens.append(merged)
    return "".join(out_tokens)

def clean_text_unified(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.strip()
    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)
    text = remove_zero_width(text)
    text = normalize_unicode(text)
    text = merge_spaced_characters_simple(text)
    text = merge_obfuscated_sequences(text)
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


# =========================
# LOAD ARTIFACTS
# =========================

def focal_loss(gamma=2., alpha=NEW_ALPHA):
    import tensorflow as tf
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        return -alpha * y_true * ((1 - y_pred) ** gamma) * tf.math.log(y_pred) - \
               (1 - alpha) * (1 - y_true) * (y_pred ** gamma) * tf.math.log(1 - y_pred)
    return loss

def load_artifacts():
    if not os.path.exists(TOKENIZER_PATH):
        raise FileNotFoundError(f"Tokenizer not found: {TOKENIZER_PATH}")

    with open(TOKENIZER_PATH, "rb") as f:
        tokenizer = pickle.load(f)

    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError(f"Model not found: {MODEL_PATH}")

    model = load_model(MODEL_PATH, custom_objects={"loss": focal_loss()})
    return model, tokenizer


# =========================
# PREDICT SINGLE / BATCH
# =========================

def predict_single(text, model, tokenizer, threshold=OPTIMAL_THRESHOLD):
    cleaned = clean_text_unified(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    prob = model.predict(pad, verbose=0)[0][0]
    label = int(prob > threshold)
    return {
        "input": text,
        "cleaned": cleaned,
        "prob": float(prob),
        "label": label
    }

def predict_batch(list_text, model, tokenizer, threshold=OPTIMAL_THRESHOLD):
    cleaned = [clean_text_unified(t) for t in list_text]
    seq = tokenizer.texts_to_sequences(cleaned)
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    probs = model.predict(pad, verbose=0).reshape(-1)
    labels = (probs > threshold).astype(int)
    result = []
    for raw, clean, prob, lbl in zip(list_text, cleaned, probs, labels):
        result.append({
            "input": raw,
            "cleaned": clean,
            "prob": float(prob),
            "label": int(lbl)
        })
    return result


# =========================
# CLI DEMO (run: python test_robust_judol_model.py)
# =========================

if __name__ == "__main__":
    model, tokenizer = load_artifacts()
    print("Artifacts loaded. Ready to test.\n")

    sample = [
         # --- JUDOL TESTS ---
    "WD 500rb di GACOR777 mantap langsung cair!",
    "buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!",
    "main slot pasti untung, depo terus bosku",
    "BOSKU WD 5 JUTA HARI INI DI SITUS RAJA88.",
    "S L 0 T P R 1 M A 8 8 8. Jp 100k wd cepat.", 
    "Cepat gabung w1n k3lub, b4rus4n w d s4ld0 5j.",
    "Lagi gacor main di maxwin888 hari ini.",
    "Top banget! Terimakasih 𝗛𝗢𝗧𝗝𝗘𝗣𝗘𝟱𝟱 GOOD JOB!🀄🥰", # Uji Kasus Perbaikan
    "WD 1O jt di BOSSku777. *lgsg cair*!", 
    "Depo 100k, WD 20juta di RAJASLOT777. Pasti JP!",
    "Aku di MANTAP888. Gacor terus!", 
    "buruan daftar di MANTAPJEPE888 promonya gede banget", 
    "Paling the best! HOTJEPE55 Memang GOOD JOB banget!",
    "main di slotvip4d aja, depo 50 wd 500", 
    "gacorwd77 adalah situs paling oke.",
    "wd 1000 jp. link ada di bio ya.", 
    "jp 2OOm jt sudah di WD. Paling gacor di sini.", 
    "S L O T G A C O R 1 2 3 . C O M. Puji Tuhan, rejeki nomplok",
    "asli keren 𝗚𝗔𝗟𝗔𝗕𝗘𝗧𝟲𝟴𝟴 selalu bikin hepyy.", 
    "tokowin303 4w1 likeReply", 
    "Saya mau ngasih saran yg baik ya bg kemaren saya main di 𝙈𝘼𝙉𝙏𝙐𝙇𝟰𝟬𝟰 di kasih 22juta", 
    "𝗟𝗔𝗕𝗔𝟵𝟭 emang the real MVP, selalu bikin kita bahagia dengan kemenangan emang the real MVP, selalu bikin kita bahagia dengan kemenangan", 
    "slotvip777 depo 50 wd 500", 
    "JP boss 44D mantap",
    "💰 WD 5🟢0🟢0k di P U L A U W I N 88! Langsung 🤑 JP!", 
    "S L 0 T G A C 0 R 1 2 3 --- jp 500k wd cepat", 
    "𝗠𝗔𝗡𝗧𝗔𝗣𝗝𝗘𝗣𝗘𝟴𝟴🔥🔥 WD 2𝟬𝟬j", 
    "W D   1 0 j t .. d i 𝗕𝗢𝗦𝗦𝗞𝘂777 !!", 
    "jp besar 💵 WD kecil. Main di s l 0 t v 1 p 1 1 1 sekarang!", 
    "p_υ_l_α_υ_w_ι_n88 💰 jp 10J cepat WD!!", 
    "garudahoki🔥 mantapjepe888 🏆 jp 100k wd lancar", 
    "wd sepuluh juta di slotvip111 🎯 langsung cair", 
    "wd cepat + JP 200k main di RAJASLOT777 sekarang", 
    "Daft4r di S G 1 ‧ 8‧8‧ bentar lagi ada drop mantap.",
    "mAin di b𝚘𝚜ﻜ𝚞7𝟽𝟽 enak bet, wd aman.",
    "bo.s ku. tujuh tujuh tujuh lagi gila2an promo wdee.",
    "m​a​i​n​ ​d​i​ ​b​o​s​k​u​7​7​7​ g​a​s​k​e​n​.",
    "Coba deh di \"QlRTT1QxMDA=\" katanya jam 3 pagi gacor.",
    "Main di cptlv888 lumayan tarikannya.",
    "我 在 博 司 苦 七星 七星 七星 刚 取 款.",
    "b0sku77? gas bet wd ngebut bgt.",
    "b o s k u\n7 7 7\n...disusun vertikal.",
    "WD 600 tadi, no cap. That site hits diff fr.",
    "M@!N di B0$KU^777 cuy wd kuzz teroosss.",
    "bo sku 7 7 7 — karena kamera burem jd gini.",

    # --- NON-JUDOL TESTS ---
    "Beli skin game baru ah, keren nih!", 
    "Ini channel YouTube favoritku, sering update video baru.",
    "Makan malam apa hari ini?",
    "Saya mau nonton film gratis di y0utube, tapi juga WD 5 ribu dari tabungan",
    "WD 50 ribu saja, buat beli kopi.", 
    "wah mantap nih, motor baru udah di WD sama pemiliknya", 
    "itu tempat bosku yang warna hijau itu loh, yg sering iklan jam 2 pagi.",
    "Tadi malem aku pecah telur di tempat sebelah. Tarikan bersih bgt.", 
    "Udah, masuk aja ke yang kemarin aku bilang. Yang tiap hari bagi hadiah itu.",
    "Wkwk mantap jp lagi di “platform terpercaya” itu katanya 🤡",
    "bos ku sudah datang tadi pagi bawa hadiah 25k. Gila.",
    "Ingat, keberanian itu modal. Kalau mau hasil besar, tempat itu udah aku rekomendasiin kemarin.",
    "Semoga sukses selalu dan sehat terus ya. Kontennya sangat bermanfaat!", 
    "Saya transfer 500 ribu ke rekening teman tadi pagi, semoga lancar.", 
    "Main Mobile Legends seru banget, tapi harus top up dulu buat beli diamond.", 
    "Saya pesan nasi Padang di warung Bossku tadi, rasanya mantap.", 
    "Saya sudah JP (Jelas Puas) sama pelayanan toko ini.", 
    "Komentar ini hanya berisi pujian, tidak ada unsur promosi atau judi.",
    "link untuk download materi kuliah ada di deskripsi video ya teman-teman.", 
    "Terima kasih atas video tutorial cara WD (Withdrawal) uang dari platform saham." ,
    "Dijamin happy udah kaya rumah kedua 𝐀𝐑𝐄𝐍𝐀𝟗𝟐𝟑 disini 👏",
    "cuma di tempat lu bang yang paling amanah🙌",
    "𝙈𝙀𝙂𝘼𝘽𝙀𝙏𝟭𝟴𝟵 emang the real MVP selalu bikin kita bahagia dengan kemenangannya",
    # Non-Judol menggunakan WD (Withdraw) dan Nominal
    "Tadi pagi aku WD (Withdraw) uang tunai dari ATM 500 ribu.",
    "WD dana beasiswa sudah cair, total 5 juta! Alhamdulillah.",
    "Jangan lupa WD hasil penjualan dari platform e-commerce itu ya.",
    
    # Non-Judol menggunakan JP (Jelas Puas, Jaminan Produk, etc.)
    "Pelayanan toko ini JP (Jelas Puas) banget, barangnya 'Gacor'!",
    "Aku JP (Jaminan Produk) 100% kalau kamu beli di Bossku Official Store.",
    "Wah, si Bossku tadi pagi datang bawa hadiah, mantap JP!",

    # Non-Judol menggunakan Gacor, Bossku, Depo (konteks non-judi)
    "Motor baruku Gacor banget, tarikannya mantap!",
    "Si Bossku (panggilan teman) tadi Depo (Deposit) makanan ke meja.",
    "Restoran ini Gacor tiap hari, ramenya nggak ketulungan.",
    
    # Non-Judol dengan Kombinasi Kata Kunci Ambigu
    "Aku dapat promo Depo 50k, dapat 50k di Toko X (bukan situs judi).",
    "Gacor banget, main game Mobile Legends dapat JP (Juara Pertama).",
    "Kata Bossku, ini adalah situs terbaik untuk beli akun game (bukan judi).",
    "WD 5 ribu aja buat beli kopi, soalnya dompet udah 'Gacor'." # Gacor = Kosong
    ]

    print("=== EXAMPLE PREDICTIONS ===")
    out = predict_batch(sample, model, tokenizer)
    for r in out:
        print(r)


Artifacts loaded. Ready to test.

=== EXAMPLE PREDICTIONS ===
{'input': 'WD 500rb di GACOR777 mantap langsung cair!', 'cleaned': 'wd 500rb di gacor777 mantap langsung cair', 'prob': 0.9957224130630493, 'label': 1}
{'input': 'buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!', 'cleaned': 'buruan daftar di mantapjp888 depo 100k langsung dapat bonus', 'prob': 0.977615237236023, 'label': 1}
{'input': 'main slot pasti untung, depo terus bosku', 'cleaned': 'main slot pasti untung depo terus bosku', 'prob': 0.6435208320617676, 'label': 0}
{'input': 'BOSKU WD 5 JUTA HARI INI DI SITUS RAJA88.', 'cleaned': 'bosku wd 5 juta hari ini di situs raja88', 'prob': 0.9661065340042114, 'label': 1}
{'input': 'S L 0 T P R 1 M A 8 8 8. Jp 100k wd cepat.', 'cleaned': 'sl0tpr1ma88 8 jp 100k wd cepat', 'prob': 0.9869669079780579, 'label': 1}
{'input': 'Cepat gabung w1n k3lub, b4rus4n w d s4ld0 5j.', 'cleaned': 'cepat gabung w1n k3lub b4rus4n w d s4ld0 5j', 'prob': 0.9956942796707153, 'label': 1}
{'